# 과제2. [평가]Mobile 고객 이탈 분석

* 여러분은 OO 통신화사 데이터분석가 입니다.
* 회사는 약정기간이 끝난 고객이 번호이동(이탈)해 가는 문제를 해결하고자 합니다.
* 그래서 여러분에게, 어떤 고객이 번호이동(이탈)해 가는지 데이터분석을 의뢰하였습니다.
* 고객 이탈여부(CHURN)에 영향을 주는 요인을 찾아 봅시다.

![](https://d18lkz4dllo6v2.cloudfront.net/cumulus_uploads/entry/23964/mobile%20phones.png)

## 0.환경설정

### 1) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.graphics.mosaicplot import mosaic
from scipy import stats as spst
import statsmodels.api as sm

### 2) 데이터 로딩

In [ ]:
# mobile data
data_path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/mobile_NA2.csv"
mobile = pd.read_csv(data_path)
mobile.drop(['id', 'REPORTED_USAGE_LEVEL','OVER_15MINS_CALLS_PER_MONTH'], axis = 1, inplace = True)
mobile.head()

|변수 명|내용|구분|
|	----	|	----	|	----	|
|	COLLEGE	|	대학졸업 여부(1,0) - 범주	|		|
|	INCOME	|	연 수입액(달러)	|		|
|	OVERAGE	|	월 초과사용 시간(분)	|		|
|	LEFTOVER	|	월 사용 잔여시간비율(%)	|		|
|	HOUSE	|	집 가격(달러)	|		|
|	HANDSET_PRICE	|	핸드폰 가격(달러)	|		|
|	AVERAGE_CALL_DURATION	|	평균 통화시간(분)	|		|
|	REPORTED_SATISFACTION	|	만족도 설문('very_unsat', 'unsat', 'avg', 'sat', 'very_sat' ) - 범주	|		|
|	CONSIDERING_CHANGE_OF_PLAN	|	변경 계획 설문('never_thought', 'no', 'perhaps', 'considering',   'actively_looking_into_it') - 범주	|		|
|	**CHURN**	|	이탈여부(1 : 이탈, 0 : 잔류)	|	**Target**	|


In [ ]:
mobile.isna().sum()

----

## 1.이변량 분석 
* 자료의 종류에 맞게 X --> Y 에 대해서 그래프(시각화)와 가설검정(수치화)를 수행하고 결과를 평가합니다.
* 가설검정시 다음의 항목을 참조하여 수행합니다.
    * 유의수준 : 5%
    * 숫자 --> 숫자 : 상관분석
    * 범주 --> 범주 : 카이제곱검정
    * 범주 --> 숫자 : t검정, 분산분석
    * 숫자 --> 범주 : 로지스틱 회귀모형을 통해, 회귀계수의 P.value로 검정을 수행합니다.

In [ ]:
target = 'CHURN'

### (1) 범주형 Feature --> CHURN
* 모든 범주형 Feature에 대해서 Y와 비교하여 차트를 그리고 수치화 하시오.  
    ① 교차표(pd.crosstab())  
    ② 시각화( Stacked Bar, Mosaic)  
    ③ 가설검정(카이제곱검정)
* 각 feature 별로 관계의 정도를 3가지로 정리해봅시다. 

In [ ]:
target = 'CHURN'

In [ ]:
# 전체 이탈율
total_mean = mobile[target].mean()
print(total_mean)

#### 1) COLLEGE --> CHURN

In [ ]:
var = 'COLLEGE'

① 교차표(pd.crosstab())

In [ ]:
temp = pd.crosstab(mobile[var], mobile[target] , normalize = 'index')
temp

② 시각화( Stacked Bar, Mosaic)

In [ ]:
# 100% Stacked Bar
temp.plot.bar(stacked=True)
plt.axhline(1-total_mean, color = 'r')
plt.show()

In [ ]:
# Mosaic
mosaic(mobile, [ var, target]) # [x축, y축]
plt.axhline(1- total_mean, color = 'r')
plt.xlabel(var)
plt.show()

시각화를 통해 파악한 내용을 적어 봅시다.

> COLLEGE와 CHURN 간에는 관련이 없어 보임

③ 가설검정(카이제곱검정)

In [ ]:
table = pd.crosstab(mobile[target], mobile[var])
chiresult = spst.chi2_contingency(table)
print("카이제곱통계량 : {}".format(chiresult[0]))
print("p-value : {:.20f}".format(chiresult[1]))

④ 시각화와 가설검정을 통해서 파악한 내용을 적어 봅시다.

> * 차트를 통해서는 대학졸업여부와 이탈여부는 관련이 없어 보임. 
* 그러나 카이제곱검정으로는 pvalue가 0.02로 유의수준 0.05보다 작게 나옴
* 결론 : 약한 관계

#### 2) REPORTED_SATISFACTION  --> CHURN

① 교차표(pd.crosstab())

② 시각화( Stacked Bar, Mosaic)

시각화를 통해 파악한 내용을 적어 봅시다.

③ 가설검정(카이제곱검정)

④ 시각화와 가설검정을 통해서 파악한 내용을 적어 봅시다.

#### 3) CONSIDERING_CHANGE_OF_PLAN --> CHURN

① 교차표(pd.crosstab())

② 시각화( Stacked Bar, Mosaic)

시각화를 통해 파악한 내용을 적어 봅시다.

③ 가설검정(카이제곱검정)

④ 시각화와 가설검정을 통해서 파악한 내용을 적어 봅시다.

### (2) 숫자형 Feature --> CHURN

* 숫자형 Feature에 대해서 Y와 비교하여 차트를 그리고 수치화 하시오.  
    ① 그래프 : histogram, densityplot   
    ② 수치화 : 로지스틱회귀

#### 1) INCOME --> CHURN


In [ ]:
var = 'INCOME'

① 그래프 : histogram, densityplot

In [ ]:
# 히스토그램
sns.histplot(x=var, data = mobile, hue = target)
plt.show()

In [ ]:

att_mean = mobile.loc[mobile['CHURN'] == 1, ''].count() / mobile.shape[0]
print(att_mean)
# sns.kdeplot(x='MonthlyIncome',  data = dataset, hue = 'Attrition', multiple = 'fill')
# plt.axhline(att_mean)
# plt.show()

In [ ]:
# density plot
t0 = mobile.loc[mobile[target]==0]
t1 = mobile.loc[mobile[target]==1]

sns.kdeplot(x=var, data = t0, label = '0')
sns.kdeplot(x=var, data = t1, label = '1')

plt.legend()
plt.show()

시각화를 통해 파악한 내용을 적어 봅시다.

> * 100000 달러를 기준으로 나뉨.
    * 100000 이내 : 잔류가 더 높음
    * 100000 이상 : 이탈이 더 높음.
* 왜???

② 수치화 : 로지스틱회귀

In [ ]:
model=sm.Logit(mobile[target], mobile[var])
result=model.fit()
print(result.pvalues)

③ 시각화와 수치화를 통해 파악한 내용을 적어 봅시다.

> * 그래프상으로 10만달러 기준으로 이탈율에 차이가 있다.
* 로지스틱회귀로 부터 얻은 P-value가 0.000044로 0.05보다 작으므로 관련이 있다.
* 결론: 강한 관계가 있다.

#### 2) OVERAGE --> CHURN


① 그래프 : histogram, densityplot

시각화를 통해 파악한 내용을 적어 봅시다.

② 수치화 : 로지스틱회귀

③ 시각화와 수치화를 통해 파악한 내용을 적어 봅시다.

#### 3) LEFTOVER --> CHURN


① 그래프 : histogram, densityplot

시각화를 통해 파악한 내용을 적어 봅시다.

② 수치화 : 로지스틱회귀

③ 시각화와 수치화를 통해 파악한 내용을 적어 봅시다.

#### 4) HOUSE --> CHURN


* **[주의]** HOUSE에 NaN이 존재합니다. ==> 로지스틱회귀를 수행하기 위해서는 NaN을 제외한 데이터로 수행해야 합니다.

① 그래프 : histogram, densityplot

시각화를 통해 파악한 내용을 적어 봅시다.

② 수치화 : 로지스틱회귀

③ 시각화와 수치화를 통해 파악한 내용을 적어 봅시다.

#### 5) HANDSET_PRICE --> CHURN


① 그래프 : histogram, densityplot

시각화를 통해 파악한 내용을 적어 봅시다.

② 수치화 : 로지스틱회귀

③ 시각화와 수치화를 통해 파악한 내용을 적어 봅시다.

#### 6) AVERAGE_CALL_DURATION --> CHURN


① 그래프 : histogram, densityplot

시각화를 통해 파악한 내용을 적어 봅시다.

② 수치화 : 로지스틱회귀

③ 시각화와 수치화를 통해 파악한 내용을 적어 봅시다.

## 2.관계 정리

### 1) 강한관계의 변수

### 2) 약한관계의 변수

### 3) 관계 없음

### 4) 추가 분석